In [1]:
print("ok")

ok


In [2]:
%pwd #project working directory

'c:\\Users\\tanko\\Individual Coding Projects\\Trading-Chatbot-Generative-AI\\research'

In [3]:
# STEP 1
import os 
os.chdir("../") #One folder back

In [4]:
%pwd # To check folder location else path issue
     # Always check in Project Folder directory 
     # (Trading-Chatbot-Generative-AI)

'c:\\Users\\tanko\\Individual Coding Projects\\Trading-Chatbot-Generative-AI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# STEP 2
# Extract Data From the PDF File

def load_pdf_file(data):
    #glob: load PDF documents
    #loader_cls: Extract information from PDF
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [7]:
# Obtain extracted information from PDF
extracted_data = load_pdf_file(data='Data\\')

In [8]:
extracted_data

[Document(metadata={'source': 'Data\\Trading_Book.pdf', 'page': 0}, page_content='THE COMPLETE \nGUIDE TO DAY\nTRADING\nA Practical Manual \nFrom A Professional Day Trading Coach\nMarkus Heitkoetter'),
 Document(metadata={'source': 'Data\\Trading_Book.pdf', 'page': 1}, page_content='The opinions expressed in this manuscript are solely the opinions of the author and do not represent  \nthe opinions or thoughts of the publisher.  The author represents and warrants that s/he either owns  \nor has the legal right to publish all material in this book.\nThe Complete Guide to Day Trading\nA Practical Manual From A Professional Day Trading Coach\nAll Rights Reserved.\nCopyright © 2008 Markus Heitkoetter\nV1.0\nEdited by Nicole Quinlan\nThis book may not be reproduced, transmitted, or stored in whole or in part by any means, including  \ngraphic, electronic, or mechanical without the express written consent of the publisher except in the  \ncase of brief quotations embodied in critical articles

In [9]:
# Split the Data into Text Chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 832


In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

c:\Users\tanko\anaconda3\envs\tradebot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'cached_download' from 'huggingface_hub' (c:\Users\tanko\anaconda3\envs\tradebot\lib\site-packages\huggingface_hub\__init__.py)

In [41]:
# Download the Embedding model from Hugging Face

# Embedding model: sentence-transformers/all-MiniLM-L6-v2
# Sentence transformer: maps sentences and paragrapgs to a 384 dimensional dense vector space
#                       and can be used for tasks like clustering / semantic search
# Vector dimension: 384
# Using Pinecone vector database -> need to mention vector dimension
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:
# Test whether model able to convert sentence to vector embedding
query_result = embeddings.embed_query("Hello World")
print("Length:", len(query_result))

In [15]:
# vector

# query_result 

In [16]:
# Method 1
# Go to PINECONE website: create API KEY (optional) to save in .env and create index

In [ ]:
# Retrieve pinecone api key from .env into the system's environment
from dotenv import load_dotenv
load_dotenv() # reads a '.env' file 

In [ ]:
# Save the API KEY inside the variable
import os
# Access the variable
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

# ISSUE FACED: ensure doing it under the project directory 

In [ ]:
# Method 2

# Create serverless
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key = PINECONE_API_KEY)

index_name = "tradebot"

pc.create_index(
    name = index_name,
    dimension = 384,
    metric = "cosine",
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1"
    )
)


In [ ]:
import os 

#Set pinecone api key as environment variable
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
# Store vector embedding 
# Embed each chunk and upsert the embeddings into your Pinecone index
from langchain.vectorstores import Pinecone

# Inside the index_name, data: text_chunks and embedding model
# Convert the data (832 chunks) into vector embedding & will be restore to Pinecone vector database
docsearch = Pinecone.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding = embeddings,
)


In [ ]:
# Load Existing index
# - load the created index and all of the loaded vector
from langchain.vectorstores import Pinecone
# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = Pinecone.from_existing_index(
    index_name = index_name,
    embedding = embeddings,
)

In [29]:
# Semantic Search: search for the same words
# search_kwargs: means give 3 relevant answers

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [30]:
# Ask Questions
# invoke(): used to pass the query to the retriever object
# Retriever object is designed to:
# - Accept a query string (like "What is stop loss?").
# - Process the query, by retrieving relevant info from a knowledge base or database.
# - Return the results, which are stored in retriever.
retriever_docs = retriever.invoke("What is stop loss?")

In [ ]:
retriever_docs

In [ ]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)